## 1. Setup & Imports

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import the Oil class
from bp_conversions import Oil

# Set style for better plots
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Imports successful!")

## 2. Basic Conversion: D86 → D2887 & TBP

Start with a simple D86 distillation curve and convert to other standards.

In [ ]:
# Sample D86 data (kerosene-like)
# Format: [volume%, temperature_C]
d86_data = [
    [0, 160],      # IBP (Initial Boiling Point)
    [10, 172],
    [30, 192],
    [50, 225],
    [70, 260],
    [90, 280],
    [100, 290]     # FBP (Final Boiling Point)
]

# Create Oil object with D86 input
# Density = 820 kg/m³ (typical kerosene)
oil = Oil(d86_data, Density=820, input_type='D86')

print(f"Input type: {oil.input_type}")
print(f"Density: {oil.Density} kg/m³")
print(f"\nAutomatic conversions created:")
print(f"  ✓ D86 interpolator")
print(f"  ✓ D2887 interpolator")
print(f"  ✓ TBP interpolator (API method)")
print(f"  ✓ TBP interpolator (Daubert method)")

## 3. Query Conversions at Any Point

The interpolators allow querying at any volume percent.

In [ ]:
# Create a table showing conversions at standard points
vol_percents = [0, 10, 30, 50, 70, 90, 100]

conversion_data = []
for vol_pct in vol_percents:
    d86_temp = oil.D86_interp(vol_pct)
    d2887_temp = oil.D2887_interp(vol_pct)
    tbp_api_temp = oil.TBP_interp(vol_pct)
    tbp_daubert_temp = oil.Daubert_TBP_interp(vol_pct)
    
    conversion_data.append({
        'Vol %': vol_pct,
        'D86 (°C)': round(d86_temp, 1),
        'D2887 (°C)': round(d2887_temp, 1),
        'TBP-API (°C)': round(tbp_api_temp, 1),
        'TBP-Daubert (°C)': round(tbp_daubert_temp, 1),
        'D86→D2887 (°C)': round(d2887_temp - d86_temp, 1),
        'D2887→TBP (°C)': round(tbp_api_temp - d2887_temp, 1),
    })

df = pd.DataFrame(conversion_data)
print("\n🔄 Conversion Results:")
print(df.to_string(index=False))

## 4. Visualize All Curves

Compare the three distillation standards visually.

In [ ]:
# Create smooth curves for plotting
vol_pcts = np.linspace(0, 100, 100)

# Get temperatures for each curve
d86_temps = [oil.D86_interp(v) for v in vol_pcts]
d2887_temps = [oil.D2887_interp(v) for v in vol_pcts]
tbp_temps = [oil.TBP_interp(v) for v in vol_pcts]

# Plot
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(vol_pcts, d86_temps, linewidth=2.5, label='D86 (ASTM D86)', marker='o', markersize=4, alpha=0.7)
ax.plot(vol_pcts, d2887_temps, linewidth=2.5, label='D2887 (SimDis)', marker='s', markersize=4, alpha=0.7)
ax.plot(vol_pcts, tbp_temps, linewidth=2.5, label='TBP (True Boiling Point)', marker='^', markersize=4, alpha=0.7)

# Formatting
ax.set_xlabel('Volume Distilled (%)', fontsize=12, fontweight='bold')
ax.set_ylabel('Temperature (°C)', fontsize=12, fontweight='bold')
ax.set_title('Distillation Curve Interconversion: D86 → D2887 → TBP', fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 100)

plt.tight_layout()
plt.show()

print("\n📊 Key observations:")
print(f"  • D86 (lowest) = ASTM D86 method with heat losses")
print(f"  • D2887 (middle) = Gas Chromatography (SimDis) method")
print(f"  • TBP (highest) = Theoretical equilibrium curve")
print(f"  • Always: D86 < D2887 < TBP")

## 5. Petroleum Properties

Calculate characterization properties automatically.

In [ ]:
# The Oil object automatically calculates important properties

print("\n⚙️ Petroleum Properties:")
print(f"\n  VABP (Volume Average Boiling Point): {oil.VABP:.1f} °F ({oil.convert_temperature(oil.VABP, 'F', 'C'):.1f} °C)")
print(f"  MeABP (Mean Average Boiling Point):   {oil.MeABP:.1f} °C ({oil.convert_temperature(oil.MeABP, 'C', 'F'):.1f} °F)")
print(f"  Watson K (Characterization Factor):  {oil.WatsonK:.3f}")
print(f"\n  Density: {oil.Density} kg/m³")

# Watson K interpretation
if oil.WatsonK < 11.5:
    oil_type = "Naphthenic (aromatic-rich)"
elif oil.WatsonK < 12.5:
    oil_type = "Mixed (balanced aromatics/paraffins)"
else:
    oil_type = "Paraffinic (alkane-rich)"

print(f"\n  Oil Characterization: {oil_type}")
print(f"    (Watson K tells us the oil's aromatic vs paraffinic nature)")

## 6. Temperature Unit Conversion

The Oil class has built-in temperature converter.

In [ ]:
# Convert temperatures between units

test_temp_c = 225  # Celsius

# Convert to other units
temp_f = oil.convert_temperature(test_temp_c, 'C', 'F')
temp_k = oil.convert_temperature(test_temp_c, 'C', 'K')
temp_r = oil.convert_temperature(test_temp_c, 'C', 'R')

print(f"\n🌡️ Temperature Conversions (example: {test_temp_c}°C)")
print(f"  {test_temp_c:.1f}°C  = {temp_f:.1f}°F")
print(f"  {test_temp_c:.1f}°C  = {temp_k:.1f}K")
print(f"  {test_temp_c:.1f}°C  = {temp_r:.1f}°R")

print(f"\n  All conversions preserve physical meaning (correlations use Rankine internally)")

## 7. Reverse Conversion: D2887 → D86 & TBP

Start with D2887 (SimDis) and convert to other standards.

In [ ]:
# D2887 input data
d2887_data = [
    [0, 165],      # D2887 is higher than D86
    [10, 180],
    [30, 198],
    [50, 230],
    [70, 265],
    [90, 285],
    [100, 295]
]

# Create Oil object with D2887 input
oil2 = Oil(d2887_data, Density=820, input_type='D2887')

# Compare with original D86
print("\n🔄 Reverse Conversion: D2887 → D86")
print("\nComparison at 50% point:")
print(f"  Original Oil (D86 input):")
print(f"    D86 @ 50%: {oil.D86_interp(50):.1f}°C")
print(f"    D2887 @ 50%: {oil.D2887_interp(50):.1f}°C")
print(f"\n  New Oil (D2887 input):")
print(f"    D86 @ 50%: {oil2.D86_interp(50):.1f}°C")
print(f"    D2887 @ 50%: {oil2.D2887_interp(50):.1f}°C")
print(f"\n  Properties:")
print(f"    VABP: {oil2.VABP:.1f}°F")
print(f"    Watson K: {oil2.WatsonK:.3f}")

## 8. Convert CSV Data

Real-world use case: Load data from CSV and convert.

In [ ]:
# In practice, you would load from CSV:
# df = pd.read_csv('my_distillation_data.csv')
# data = df[['VolPercent', 'TemperatureC']].values.tolist()

# For this example, create sample data
sample_data = [
    [0, 160], [10, 172], [20, 182], [30, 192], [40, 208],
    [50, 225], [60, 242], [70, 260], [80, 273], [90, 280], [100, 290]
]

# Create Oil object
oil3 = Oil(sample_data, Density=820, input_type='D86')

# Create results table
results = []
for vol_pct in range(0, 101, 10):
    results.append({
        'VolPercent': vol_pct,
        'D86_C': round(oil3.D86_interp(vol_pct), 1),
        'D2887_C': round(oil3.D2887_interp(vol_pct), 1),
        'TBP_C': round(oil3.TBP_interp(vol_pct), 1),
    })

results_df = pd.DataFrame(results)
print("\n📊 Example Results (Export-Ready):")
print(results_df.to_string(index=False))

# Export to CSV
# results_df.to_csv('conversion_results.csv', index=False)
print("\n💾 Save with: results_df.to_csv('conversion_results.csv', index=False)")

## 9. Batch Processing Multiple Samples

In [ ]:
# Process multiple oil samples at once

samples = {
    'Light Crude': {
        'data': [[0, 155], [50, 220], [100, 285]],
        'density': 750,
    },
    'Medium Crude': {
        'data': [[0, 160], [50, 225], [100, 290]],
        'density': 820,
    },
    'Heavy Crude': {
        'data': [[0, 170], [50, 235], [100, 305]],
        'density': 900,
    },
}

# Process all samples
batch_results = []

for sample_name, sample_info in samples.items():
    oil_sample = Oil(sample_info['data'], sample_info['density'], 'D86')
    
    batch_results.append({
        'Sample': sample_name,
        'Density (kg/m³)': sample_info['density'],
        'VABP (°F)': round(oil_sample.VABP, 1),
        'Watson K': round(oil_sample.WatsonK, 3),
        'D86@50% (°C)': round(oil_sample.D86_interp(50), 1),
        'D2887@50% (°C)': round(oil_sample.D2887_interp(50), 1),
        'TBP@50% (°C)': round(oil_sample.TBP_interp(50), 1),
    })

batch_df = pd.DataFrame(batch_results)
print("\n📋 Batch Processing Results:")
print(batch_df.to_string(index=False))

## 10. Key Takeaways

Summary and best practices.

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║     Distillation Curve Interconversion - Key Takeaways          ║
╚════════════════════════════════════════════════════════════════╝

✅ QUICK START (3 lines):
   from bp_conversions import Oil
   oil = Oil(data, density=820, input_type='D86')
   d2887_temp = oil.D2887_interp(50)  # D2887 at 50% vol

📋 TYPICAL WORKFLOW:
   1. Load data: CSV file with Vol% and Temperature
   2. Create Oil object: Oil(data, density, input_type)
   3. Query: oil.D86_interp(vol%), oil.D2887_interp(vol%), etc.
   4. Export: Save results to CSV/Excel

🔑 KEY FACTS:
   • Always: D86 < D2887 < TBP (thermodynamic order)
   • D86 is 3-7°C lower than D2887 (heat losses)
   • TBP is 0.5-2°C higher than D2887 (theoretical equilibrium)
   • Watson K: <11.5 (aromatic), 11.5-12.5 (mixed), >12.5 (paraffinic)
   • Minimum 3 data points required
   • Density range: 600-1200 kg/m³

🛠️ PROPERTIES CALCULATED:
   • VABP (Volume Average Boiling Point) in °F
   • MeABP (Mean Average Boiling Point) in °C
   • Watson K (Characterization Factor)
   • Temperature conversions (C ↔ F ↔ K ↔ R)

💡 BEST PRACTICES:
   1. Use 8 standard points: 0%, 10%, 30%, 50%, 70%, 90%, 95%, 100%
   2. Ensure monotonic temperatures (always increasing)
   3. Validate density is reasonable (600-1200 kg/m³)
   4. Check physical order: D86 < D2887 < TBP
   5. Round-trip test: D86 → D2887 → D86 (should match ±2°C)

📖 FURTHER LEARNING:
   • See QUICK_REFERENCE.md for API details
   • See ARCHITECTURE.md for technical deep-dive
   • Check test_bp_conversions.py for more examples

""")

---

**Now you're ready to use the distillation curve converter!** 🚀

### Next Steps:
1. **Use the GUI:** `python distillation_converter_gui.py`
2. **Use the API:** Import `Oil` class in your scripts
3. **Read docs:** Check `QUICK_REFERENCE.md` for all available methods
4. **Run tests:** `pytest tests/test_bp_conversions.py -v`

Happy converting! 🧪